## Preparing Security Settings

### Security Policy

In [22]:
import json
import boto3

oss_client = boto3.client('opensearchserverless')
collection_name = "text2sql-collection"

network_security_policy = json.dumps([{
    "Rules": [
        {
            "Resource": [
                f"collection/{collection_name}"
            ],
            "ResourceType": "dashboard"
        },
        {
            "Resource": [
                f"collection/{collection_name}"
            ],
            "ResourceType": "collection"
        }
    ],
    "AllowFromPublic": True
}], indent=2)


In [ ]:
try:
    oss_client.create_security_policy(
        description='Encryption policy for text2sql-collection',
        name='text2sql-security-policy',
        policy=network_security_policy,
        type='network'
    )
    print("Security policy created successfully.")
except oss_client.exceptions.ConflictException:
    print("Security policy already exists. Skipping creation.")

### Encryption Policy

In [26]:
encryption_policy = json.dumps(
    {
        'Rules': [{'Resource': ['collection/' + collection_name],
                    'ResourceType': 'collection'}],
        'AWSOwnedKey': True
    }
)

In [ ]:
try:
    oss_client.create_security_policy(
        description='Encryption policy for text2sql-collection',
        name='text2sql-encryption-policy',
        policy=encryption_policy,
        type='encryption'
    )
    print("Encryption policy created successfully.")
except oss_client.exceptions.ConflictException:
    print("Encryption policy already exists. Skipping creation.")

### Data Access Policy

In [11]:
sts_client = boto3.client('sts')
response = sts_client.get_caller_identity()

principal_arn = response['Arn']

data_access_policy = json.dumps([
      {
        "Rules": [
          {
            "Resource": [
              f"collection/{collection_name}"
            ],
            "Permission": [
              "aoss:CreateCollectionItems",
              "aoss:DeleteCollectionItems",
              "aoss:UpdateCollectionItems",
              "aoss:DescribeCollectionItems"
            ],
            "ResourceType": "collection"
          },
          {
            "Resource": [
              f"index/{collection_name}/*"
            ],
            "Permission": [
              "aoss:CreateIndex",
              "aoss:DeleteIndex",
              "aoss:UpdateIndex",
              "aoss:DescribeIndex",
              "aoss:ReadDocument",
              "aoss:WriteDocument"
            ],
            "ResourceType": "index"
          }
        ],
        "Principal": [
          f"{principal_arn}"
        ],
        "Description": "data-access-rule"
      }
    ], indent=2)

In [ ]:
try:
    oss_client.create_access_policy(
        description='Access policy for text2sql-collection',
        name='text2sql-access-policy',
        policy=data_access_policy,
        type='data'
    )
    print("Data access policy created successfully.")
except oss_client.exceptions.ConflictException:
    print("Data access  policy already exists. Skipping creation.")

## Creating OpenSearch Serverless Collection

In [ ]:
def get_collection_status(client, collection_id):
    response = client.batch_get_collection(ids=[collection_id])
    return response['collectionDetails'][0]['status']


existing_collection = oss_client.batch_get_collection(names=[collection_name])
if existing_collection['collectionDetails']:
    print(f"Collection '{collection_name}' already exists.")
    collection_detail = existing_collection['collectionDetails'][0]
else:
    response = oss_client.create_collection(
                name=collection_name,
                description="A collection for vector search operations",
                type="VECTORSEARCH",
                standbyReplicas='DISABLED'
            )
    collection_detail = response['createCollectionDetail']
    print(f"Collection '{collection_name}' created successfully.")

collection_id = collection_detail['id']
collection_arn = collection_detail['arn']

In [ ]:
import time

print("\nWaiting for collection to become active...")
while True:
    status = get_collection_status(oss_client, collection_id)
    print(f"Current status: {status}")
    if status == 'ACTIVE':
        print("Collection is now active!")
        break
    elif status == 'FAILED':
        print("Collection creation failed.")
        exit(1)
    time.sleep(30)

final_collection = oss_client.batch_get_collection(ids=[collection_id])
final_detail = final_collection['collectionDetails'][0]

collection_endpoint = final_detail.get('collectionEndpoint', 'Not available')
dashboard_endpoint = final_detail.get('dashboardEndpoint', 'Not available')
collection_status = final_detail['status']

print("\nFinal Collection Details:")
print(f"Collection Name: {collection_name}")
print(f"Collection ID: {collection_id}")
print(f"Collection ARN: {collection_arn}")
print(f"Collection Endpoint: {collection_endpoint}")
print(f"Dashboard Endpoint: {dashboard_endpoint}")
print(f"Collection Status: {collection_status}")

In [ ]:
%store collection_name collection_arn collection_id collection_endpoint dashboard_endpoint collection_status

In [33]:
!pip install -q python-dotenv

In [ ]:
from dotenv import load_dotenv, set_key
import os

current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
env_path = os.path.join(parent_path, '.env')

load_dotenv(dotenv_path=env_path)

variables = {
    'COLLECTION_NAME': collection_name,
    'COLLECTION_ARN': collection_arn,
    'COLLECTION_ID': collection_id,
    'COLLECTION_ENDPOINT': collection_endpoint,
    'DASHBOARD_ENDPOINT': dashboard_endpoint,
    'COLLECTION_STATUS': collection_status
}

for key, value in variables.items():
    set_key(env_path, key, str(value))

print(f"Variables have been stored in .env file at {env_path}")